In [29]:
import os
import pandas as pd

# Path to the participant folder (e.g., "Participant 1")
parent_dir = "C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9"
output_dir = os.path.join(parent_dir, "cleaned")

# Create the 'cleaned' subfolder if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Identify date folders like '2023-12-24'
day_folders = sorted([
    f for f in os.listdir(parent_dir)
    if os.path.isdir(os.path.join(parent_dir, f)) and f.startswith("2023")
])

for day in day_folders:
    day_path = os.path.join(parent_dir, day)
    daily_dfs = []

    for root, dirs, files in os.walk(day_path):
        for file in files:
            if file.endswith(".csv"):
                file_path = os.path.join(root, file)
                try:
                    df = pd.read_csv(file_path)
                    daily_dfs.append(df)
                except Exception as e:
                    print(f"❌ Failed to read {file_path}: {e}")

    if not daily_dfs:
        print(f"⚠️ No CSV files found in {day}")
        continue

    # Merge and clean
    merged_df = pd.concat(daily_dfs, axis=1)
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
    merged_df = merged_df.dropna(axis=1, how='all')

    # Drop optional columns if they exist
    merged_df = merged_df.drop(columns=[
        col for col in ['timestamp_unix', 'participant_full_id'] if col in merged_df.columns
    ])

    # Convert timestamp and extract hour/minute
    if 'timestamp_iso' in merged_df.columns:
        try:
            merged_df['timestamp_iso'] = pd.to_datetime(merged_df['timestamp_iso'], errors='coerce')
            merged_df['hour'] = merged_df['timestamp_iso'].dt.hour
            merged_df['minute'] = merged_df['timestamp_iso'].dt.minute
        except Exception as e:
            print(f"⚠️ Failed to parse timestamp for {day}: {e}")

    # Save cleaned file
    output_path = os.path.join(output_dir, f"cleaned_{day}.csv")
    merged_df.to_csv(output_path, index=False)
    print(f"✅ Saved: {output_path}")


✅ Saved: C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9\cleaned\cleaned_2023-12-16.csv
✅ Saved: C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9\cleaned\cleaned_2023-12-17.csv
✅ Saved: C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9\cleaned\cleaned_2023-12-18.csv
✅ Saved: C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9\cleaned\cleaned_2023-12-19.csv
✅ Saved: C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9\cleaned\cleaned_2023-12-20.csv
✅ Saved: C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9\cleaned\cleaned_2023-12-21.csv
✅ Saved: C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 9\cleaned\cleaned_2023-12-22.csv


In [5]:
import pandas as pd
df = pd.read_csv("C:/Users/lpnhu/Downloads/Participant 1/cleaned/cleaned_2023-12-30.csv")
df.head()

,timestamp_iso,accelerometers_std_g,missing_value_reason,counts_x_axis,counts_y_axis,counts_z_axis,vector_magnitude,activity_class,activity_counts,activity_intensity,...,met,prv_rmssd_ms,pulse_rate_bpm,respiratory_rate_brpm,sleep_detection_stage,step_counts,temperature_celsius,wearing_detection_percentage,hour,minute
0,2023-12-30 00:00:00+00:00,0.106,NaN,2972.0,1761.0,2044.0,4013.0,walking,143.0,LPA,...,2.92,NaN,94.0,NaN,0.0,70.0,29.53,100.0,0,0
1,2023-12-30 00:01:00+00:00,0.145,NaN,8060.0,6792.0,3554.0,11123.0,generic,123.0,VPA,...,7.36,NaN,90.0,NaN,0.0,34.0,29.57,100.0,0,1
2,2023-12-30 00:02:00+00:00,0.175,NaN,7265.0,5233.0,4571.0,10052.0,generic,133.0,VPA,...,7.29,NaN,91.0,NaN,0.0,58.0,29.67,100.0,0,2
3,2023-12-30 00:03:00+00:00,0.192,NaN,5964.0,3161.0,3881.0,7786.0,generic,136.0,VPA,...,6.66,NaN,102.0,NaN,0.0,62.0,29.89,100.0,0,3
4,2023-12-30 00:04:00+00:00,0.132,NaN,5692.0,3881.0,5370.0,8734.0,generic,150.0,MPA,...,3.53,NaN,100.0,NaN,0.0,81.0,29.86,100.0,0,4


### Check NA values in cleaned files before moving on


In [31]:
import os
import pandas as pd

cleaned_dir = "C:/Users/lpnhu/Downloads/Stress_Testing_Analysis/Participant 1/cleaned"
na_summary = []

# Loop through each cleaned daily file
for file in sorted(os.listdir(cleaned_dir)):
    if file.endswith(".csv"):
        file_path = os.path.join(cleaned_dir, file)
        df = pd.read_csv(file_path)
        
        # Calculate % missing for each column
        na_percent = df.isna().mean() * 100
        na_row = na_percent.round(1).to_dict()
        na_row["file"] = file
        na_summary.append(na_row)

# Convert to DataFrame
na_df = pd.DataFrame(na_summary)
na_df.set_index("file", inplace=True)

# Show columns with highest average NA %
avg_na = na_df.mean().sort_values(ascending=False)

# Save for inspection
na_df.to_csv("participant1_na_summary.csv")
print("✅ Saved NA summary to participant1_na_summary.csv")

# Preview worst offenders
print("🔍 Columns with highest average missingness:")
print(avg_na.head(10))


✅ Saved NA summary to participant1_na_summary.csv
🔍 Columns with highest average missingness:
001_001_prv                                   97.800000
prv_prv_rmssd_ms                              97.800000
001_prv_prv_rmssd_ms                          97.800000
001_respiratory-rate_respiratory_rate_brpm    96.500000
001_001_respiratory-rate                      96.500000
respiratory-rate_respiratory_rate_brpm        96.500000
prv_rmssd_ms                                  90.083333
respiratory_rate_brpm                         86.483333
wearing-detection_missing_reason              76.500000
001_001_wearing-detection_missing_reason      76.500000
dtype: float64


In [7]:
df = pd.read_csv('participant1_na_summary.csv')
df

,file,timestamp_iso,accelerometers_std_g,missing_value_reason,counts_x_axis,counts_y_axis,counts_z_axis,vector_magnitude,activity_class,activity_counts,...,001_001_eda_missing_reason,001_001_eda,001_activity-classification_missing_reason,001_body-position_missing_reason,001_step-counts_step_counts,001_eda_eda_scl_usiemens,001_body-position_body_position_left,001_actigraphy-counts_counts_y_axis,001_pulse-rate_pulse_rate_bpm,001_001_sleep-detection
0,cleaned_2023-12-24.csv,0.0,99.9,0.1,99.9,99.9,99.9,99.9,99.9,99.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,cleaned_2023-12-25.csv,0.0,5.6,94.4,5.6,5.6,5.6,5.6,5.6,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cleaned_2023-12-26.csv,0.0,27.2,72.8,27.2,27.2,27.2,27.2,27.2,27.2,...,72.8,27.2,72.8,72.8,27.2,27.2,27.2,27.2,27.2,27.2
3,cleaned_2023-12-27.csv,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,cleaned_2023-12-28.csv,0.0,6.5,93.5,6.5,6.5,6.5,6.5,6.5,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cleaned_2023-12-29.csv,0.0,56.5,43.5,56.5,56.5,56.5,56.5,56.5,56.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,cleaned_2023-12-30.csv,0.0,52.2,47.8,52.2,52.2,52.2,52.2,52.2,52.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Drop the 'file' column (row index) and keep just the % NA values
na_pct = df.drop(columns='file').mean().sort_values()

# Filter to keep only features with < 30% missing on average
low_missing = na_pct[na_pct < 30.0]

print("✅ Features with < 30% missingness across days:")
print(low_missing)

✅ Features with < 30% missingness across days:
timestamp_iso                                          0.0
hour                                                   0.0
minute                                                 0.0
001_001_prv_missing_reason                             2.2
prv_missing_reason                                     2.2
001_prv_missing_reason                                 2.2
respiratory-rate_missing_reason                        3.5
001_001_respiratory-rate_missing_reason                3.5
001_respiratory-rate_missing_reason                    3.5
001_001_wearing-detection                             23.5
001_wearing-detection_wearing_detection_percentage    23.5
wearing-detection_wearing_detection_percentage        23.5
001_actigraphy-counts_vector_magnitude                27.2
temperature_temperature_celsius                       27.2
001_activity-intensity_activity_intensity             27.2
001_pulse-rate_pulse_rate_bpm                         27.2
step-coun